In [12]:
import cv2
import numpy as np

def msrcr_enhancement(image, sigma_list=(15, 80, 250)):

    image = image.astype(np.float32)
    log_image = np.log1p(image)
    enhanced_image = np.zeros_like(image)

    for sigma in sigma_list:
        blurred_image = cv2.GaussianBlur(log_image, (0, 0), sigma)
        difference_image = log_image - blurred_image
        scaled_image = difference_image * (128.0 / np.max(difference_image))
        restored_image = log_image + scaled_image
        enhanced_image += np.expm1(restored_image)

    enhanced_image = np.clip(enhanced_image, 0, 255).astype(np.uint8)
    return enhanced_image

def soft_nms(boxes, scores, threshold=0.3, sigma=0.5):
    selected_boxes = []
    selected_scores = []

    while boxes:
        max_score_index = np.argmax(scores)
        max_score_box = boxes[max_score_index]
        max_score = scores[max_score_index]

        selected_boxes.append(max_score_box)
        selected_scores.append(max_score)

        del boxes[max_score_index]
        del scores[max_score_index]

        for i in range(len(boxes)):
            iou = calculate_iou(max_score_box, boxes[i])
            if iou > threshold:
                scores[i] *= np.exp(-np.square(iou) / sigma)

    return selected_boxes, selected_scores

def calculate_iou(box1, box2):
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])

    intersection = max(0, x2 - x1 + 1) * max(0, y2 - y1 + 1)
    area1 = (box1[2] - box1[0] + 1) * (box1[3] - box1[1] + 1)
    area2 = (box2[2] - box2[0] + 1) * (box2[3] - box2[1] + 1)
    union = area1 + area2 - intersection

    iou = intersection / union
    return iou

def detect_faces(image_path):
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    try:
        image = cv2.imread(image_path)
        if image is None:
            print("Error: Unable to load image.")
            return
    except Exception as e:
        print("Error:", e)
        return

    enhanced_image = msrcr_enhancement(image)
    gray = cv2.cvtColor(enhanced_image, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    boxes = [(x, y, x+w, y+h) for (x, y, w, h) in faces]
    scores = [1.0] * len(boxes)

    selected_boxes, selected_scores = soft_nms(boxes, scores)

    for (x, y, x2, y2), score in zip(selected_boxes, selected_scores):
        cv2.rectangle(enhanced_image, (x, y), (x2, y2), (255, 0, 0), 2)

    cv2.imshow("Original Image", image)
    cv2.imshow("Enhanced Image", enhanced_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Example usage
image_path = './dataset/5.png'  # Update with your image path
detect_faces(image_path)


C:\Users\Sabis\AppData\Local\Temp\ipykernel_14180\3918077880.py:15: RuntimeWarning: overflow encountered in expm1
  enhanced_image += np.expm1(restored_image)
C:\Users\Sabis\AppData\Local\Temp\ipykernel_14180\3918077880.py:15: RuntimeWarning: overflow encountered in add
  enhanced_image += np.expm1(restored_image)
